# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [96]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [101]:
#1st try
csv_path = os.path.join("..", "output_data", "clean_city_data.csv")

clean_city_data_df = pd.read_csv(csv_path)
clean_city_data_df

,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country_Code
0,kloulklubed,5.01245,132.32868,86.00,70,75,6.93,pw
1,cabedelo,-5.01245,-29.07221,73.40,94,5,5.82,br
2,albany,-56.13944,116.28884,48.00,74,6,3.51,au
3,hilo,12.02988,-147.36603,80.60,65,20,6.93,us
4,hofn,64.15936,-12.02988,47.71,59,90,4.21,is
...,...,...,...,...,...,...,...,...
469,san juan,22.05478,-66.16434,62.65,30,32,8.46,us
470,hammerfest,71.17679,23.05727,42.80,93,75,3.36,no
471,tsumeb,-19.04731,18.04482,60.15,28,0,7.31,na
472,takoradi,-1.00249,1.00249,75.15,88,100,10.18,gh


In [102]:
# clean_city_data_df = clean_city_data_df.loc[(clean_city_data_df["Longitude"]>-180) & (clean_city_data_df["Longitude"]<180)]
# clean_city_data_df = clean_city_data_df.loc[(clean_city_data_df["Latitude"]>-90) & (clean_city_data_df["Latitude"]<90)]

print(f"Min longitude: {clean_city_data_df['Longitude'].min()}")
print(f"Max longitude: {clean_city_data_df['Longitude'].max()}")
print(f"Min latitude: {clean_city_data_df['Latitude'].min()}")
print(f"Max latitude: {clean_city_data_df['Latitude'].max()}")

Min longitude: -179.44571000000005
Max longitude: 179.44571000000005
Min latitude: -88.21911999999999
Max latitude: 89.22161000000001


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [103]:
# SET UP LAT LONG PAIRS
Coords1 = clean_city_data_df[['Latitude', 'Longitude']]

# Set up a list of humidities
Humidity = clean_city_data_df['Humidity']
Humidity

# CONFIGURE GMAPS
gmaps.configure(api_key=g_key)

# Customize Layout
figure_layout = {
    'width': '800px',
    'height': '600px'
}

# store the figure in a variable so we can continue to add layers to the same map
Myfig = gmaps.figure(layout=figure_layout)

# Create a marker layer using our coordinates
heat_layer = gmaps.heatmap_layer(Coords1, weights=Humidity)

# Add the layer to the map
Myfig.add_layer(heat_layer)

# display the figure with the newly added layer
Myfig 

Figure(layout=FigureLayout(height='600px', width='800px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [104]:
ideal_weather_cities_df = clean_city_data_df.loc[(clean_city_data_df["Max_Temp"]<85) & (clean_city_data_df["Max_Temp"]>65) & 
                                                        (clean_city_data_df["Wind_Speed"]<15) & (clean_city_data_df["Cloudiness"]<10) & 
                                                        (clean_city_data_df["Humidity"]<55)]
ideal_weather_cities_df

,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country_Code
38,abu dhabi,22.05478,53.13197,84.20,26,0,9.17,ae
64,sao domingos,-13.03237,-46.11454,74.16,43,1,14.36,br
87,ada,35.08715,-96.23904,69.01,39,1,3.36,us
108,yeppoon,-21.05229,154.38346,80.60,39,2,10.29,au
161,camapua,-19.04731,-54.13446,75.07,26,0,5.57,br
177,awjilah,27.06723,20.04980,81.30,33,2,9.37,ly
203,komsomolskiy,67.16683,171.42579,73.40,28,0,2.24,ru
209,xinzhou,38.09462,113.28137,65.25,31,1,3.11,cn
243,corinto,-18.04482,-44.10956,73.00,47,0,7.09,br
257,georgiyevka,43.10707,74.18426,68.00,37,0,8.95,kz


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [157]:
hotel_df = ideal_weather_cities_df.copy()
hotel_df["Hotel Name"] = ""
hotel_df["Hotel_Lat"] = ""
hotel_df["Hotel_Long"] = ""
hotel_df

,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country_Code,Hotel Name,Hotel_Lat,Hotel_Long
38,abu dhabi,22.05478,53.13197,84.20,26,0,9.17,ae,,,
64,sao domingos,-13.03237,-46.11454,74.16,43,1,14.36,br,,,
87,ada,35.08715,-96.23904,69.01,39,1,3.36,us,,,
108,yeppoon,-21.05229,154.38346,80.60,39,2,10.29,au,,,
161,camapua,-19.04731,-54.13446,75.07,26,0,5.57,br,,,
177,awjilah,27.06723,20.04980,81.30,33,2,9.37,ly,,,
203,komsomolskiy,67.16683,171.42579,73.40,28,0,2.24,ru,,,
209,xinzhou,38.09462,113.28137,65.25,31,1,3.11,cn,,,
243,corinto,-18.04482,-44.10956,73.00,47,0,7.09,br,,,
257,georgiyevka,43.10707,74.18426,68.00,37,0,8.95,kz,,,


In [159]:
# geocoordinates for the base of our search
target_location = "29.714033,-95.406480" # coordinates for Rice University

# distance, IN METERS, within which the place results must live
target_radius = 60000

# keyword term to be matched against all of Google's indexed content
target_keyword = "Hotel"

# type of establishment that we'd like to use to filter our place results
target_type = "Hotel"


# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# set up a dictionary to store all query parameters
parameters = {
#     "location": target_location,
    "keyword": target_keyword,
    "radius": target_radius,
#     "rankby": "distance"
    "type": target_type,
    "key": g_key
}

# # # run a request using our params dictionary

# run a request using our params dictionary
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    print(f"Searching for hotel in city with lat lng {lat}, {lng}...")
# update params dictionary
    parameters['location'] = f"{lat},{lng}"
   
    hotel_info = requests.get(base_url, params=parameters)
    hotel_info_json = hotel_info.json()
    # Incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, 'Hotel Name'] =  hotel_info_json['results'][0]['name']
        hotel_df.loc[index, 'Hotel_Lat'] =  hotel_info_json['results'][0]['geometry']['location']['lat']
        hotel_df.loc[index, 'Hotel_Long'] =  hotel_info_json['results'][0]['geometry']['location']['lng']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

Searching for hotel in city with lat lng 22.054779999999997, 53.131969999999995...
Missing field/result... skipping.
Searching for hotel in city with lat lng -13.032370000000002, -46.11454000000001...
Searching for hotel in city with lat lng 35.08715, -96.23904000000002...
Searching for hotel in city with lat lng -21.052290000000006, 154.38346...
Missing field/result... skipping.
Searching for hotel in city with lat lng -19.047310000000003, -54.13446...
Searching for hotel in city with lat lng 27.06723, 20.0498...
Missing field/result... skipping.
Searching for hotel in city with lat lng 67.16683, 171.42578999999998...
Missing field/result... skipping.
Searching for hotel in city with lat lng 38.094620000000006, 113.28137...
Searching for hotel in city with lat lng -18.04482, -44.109559999999995...
Searching for hotel in city with lat lng 43.10707000000001, 74.18426000000002...
Searching for hotel in city with lat lng 21.052290000000006, -80.1992...
Missing field/result... skipping.
Se

,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country_Code,Hotel Name,Hotel_Lat,Hotel_Long,Country
38,abu dhabi,22.05478,53.13197,84.20,26,0,9.17,ae,,,,ae
64,sao domingos,-13.03237,-46.11454,74.16,43,1,14.36,br,Aurora Esplendor,-12.7142,-46.4062,br
87,ada,35.08715,-96.23904,69.01,39,1,3.36,us,Holiday Inn Express & Suites Ada,34.7897,-96.6628,us
108,yeppoon,-21.05229,154.38346,80.60,39,2,10.29,au,,,,au
161,camapua,-19.04731,-54.13446,75.07,26,0,5.57,br,Hotel Zen Brasil,-19.3964,-54.5685,br
177,awjilah,27.06723,20.04980,81.30,33,2,9.37,ly,,,,ly
203,komsomolskiy,67.16683,171.42579,73.40,28,0,2.24,ru,,,,ru
209,xinzhou,38.09462,113.28137,65.25,31,1,3.11,cn,Eden International Hotel,37.6841,112.745,cn
243,corinto,-18.04482,-44.10956,73.00,47,0,7.09,br,Águas de Santa Bárbara Resort Hotel,-18.035,-44.0738,br
257,georgiyevka,43.10707,74.18426,68.00,37,0,8.95,kz,Sheraton Bishkek,42.8749,74.5907,kz


In [160]:
# Clean dataframe
hotel_df['Country'] = hotel_df['Country_Code']
# --------------------------------------------------------------------------------------
# hotel_df = hotel_df.dropna(how='any')
# hotel_df = hotel_df[~hotel_df.Max_Temp.str.contains(" ", na=False)]
# ----------------------------------------------------------------------
# hotel_df = hotel_df.loc[(hotel_df["Hotel_Lat"]=="")]
# hotel_df = hotel_df.loc[(hotel_df["Hotel_Long"]=="")]
# --------------------------------------------------------------------------------------
hotel_df

,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country_Code,Hotel Name,Hotel_Lat,Hotel_Long,Country
38,abu dhabi,22.05478,53.13197,84.20,26,0,9.17,ae,,,,ae
64,sao domingos,-13.03237,-46.11454,74.16,43,1,14.36,br,Aurora Esplendor,-12.7142,-46.4062,br
87,ada,35.08715,-96.23904,69.01,39,1,3.36,us,Holiday Inn Express & Suites Ada,34.7897,-96.6628,us
108,yeppoon,-21.05229,154.38346,80.60,39,2,10.29,au,,,,au
161,camapua,-19.04731,-54.13446,75.07,26,0,5.57,br,Hotel Zen Brasil,-19.3964,-54.5685,br
177,awjilah,27.06723,20.04980,81.30,33,2,9.37,ly,,,,ly
203,komsomolskiy,67.16683,171.42579,73.40,28,0,2.24,ru,,,,ru
209,xinzhou,38.09462,113.28137,65.25,31,1,3.11,cn,Eden International Hotel,37.6841,112.745,cn
243,corinto,-18.04482,-44.10956,73.00,47,0,7.09,br,Águas de Santa Bárbara Resort Hotel,-18.035,-44.0738,br
257,georgiyevka,43.10707,74.18426,68.00,37,0,8.95,kz,Sheraton Bishkek,42.8749,74.5907,kz


In [161]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [162]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations)
Myfig.add_layer(hotel_layer)

# Display figure
Myfig


Figure(layout=FigureLayout(height='600px', width='800px'))